# 🦜⛓️ Langchain Experiments - To Share 

## 🧮🧰 Mini Code Interpreter Tool

In [46]:
from langchain.agents import Tool

# evaluate python code
def evalPython(input: str) -> str:
    # print("*****************************************")
    # print(input)
    # print("*****************************************")
    try:
        namespace = {}
        exec(input, namespace)
        result = namespace['result']
        # print("*****************************************")
        # print(result)
        # print("*****************************************")
        return result

    except Exception as e:
        return e

codeInterpreterTool = Tool(
        name='Python code interpreter',
        func= lambda string: evalPython(string),
        description="""Python code interpreter is a tool that allows to perform operations in Python to solve problem.
 'Action Input' are lines of Python code that will be evaluated, the result will be returned as 'Action Output'.
 Make sure the final variable that is the result you want to return is named 'result' in the code.""",
        # return_direct=True,
    )



> Entering new AgentExecutor chain...
I need to write a GraphQL query to retrieve the names of all ghosts younger than 100.
Action: query_graphql
Action Input: query {
  ghosts {
    name
    age
  }
}
Observation: "{\n  \"ghosts\": [\n    {\n      \"name\": \"Casper\",\n      \"age\": 70\n    },\n    {\n      \"name\": \"Slimer\",\n      \"age\": 30\n    },\n    {\n      \"name\": \"The Grey Lady\",\n      \"age\": 200\n    },\n    {\n      \"name\": \"The Headless Horseman\",\n      \"age\": 300\n    },\n    {\n      \"name\": \"The Bell Witch\",\n      \"age\": 180\n    },\n    {\n      \"name\": \"The Brown Lady of Raynham Hall\",\n      \"age\": 300\n    },\n    {\n      \"name\": \"The Bride in Black\",\n      \"age\": 50\n    },\n    {\n      \"name\": \"The Chained Oak Ghost\",\n      \"age\": 150\n    },\n    {\n      \"name\": \"The Flying Dutchman\",\n      \"age\": 400\n    },\n    {\n      \"name\": \"The Phantom of the Opera\",\n      \"age\": 150\n    },\n    {\n      

## 📈🛢️ GraphQL Tool With Schema Loading

In [ ]:
import json
from typing import Any
from typing import Optional

from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)
from langchain.tools.base import BaseTool
from langchain.utilities.graphql import GraphQLAPIWrapper

from langchain.utilities import GraphQLAPIWrapper
from langchain.tools.base import BaseTool
from typing import Any


class BaseGraphQLTool(BaseTool):
    """Base tool for querying a GraphQL API."""

    schema_query = """fragment FullType on __Type {
  kind
  name
  fields(includeDeprecated: true) {
    name
    args {
      ...InputValue
    }
    type {
      ...TypeRef
    }
    isDeprecated
    deprecationReason
  }
  inputFields {
    ...InputValue
  }
  interfaces {
    ...TypeRef
  }
  enumValues(includeDeprecated: true) {
    name
    isDeprecated
    deprecationReason
  }
  possibleTypes {
    ...TypeRef
  }
}
fragment InputValue on __InputValue {
  name
  type {
    ...TypeRef
  }
  defaultValue
}
fragment TypeRef on __Type {
  kind
  name
  ofType {
    kind
    name
    ofType {
      kind
      name
      ofType {
        kind
        name
        ofType {
          kind
          name
          ofType {
            kind
            name
            ofType {
              kind
              name
              ofType {
                kind
                name
              }
            }
          }
        }
      }
    }
  }
}
query IntrospectionQuery {
  __schema {
    queryType {
      name
    }
    mutationType {
      name
    }
    types {
      ...FullType
    }
    directives {
      name
      locations
      args {
        ...InputValue
      }
    }
  }
}"""

    graphql_wrapper: GraphQLAPIWrapper

    name = "query_graphql"
    description = """\
    Input to this tool is a detailed and correct GraphQL query, output is a result from the API.
    If the query is not correct, an error message will be returned.
    If an error is returned with 'Bad request' in it, rewrite the query and try again.
    If an error is returned with 'Unauthorized' in it, do not try again, but tell the user to change their authentication.
    Make sure not to add unecessary quotes to the query, as this will cause an error.

    Schema:
    """  # noqa: E501

    class Config:
        """Configuration for this pydantic object."""

        arbitrary_types_allowed = True

    def __init__(self, **kwargs: Any):
        super().__init__(**kwargs)
        schema = str(self.graphql_wrapper.run(self.schema_query)).replace('{', '{{').replace('}', '}}')
        self.description = self.description + schema + """
        
Example Input: query {{ allUsers {{ id, name, email }} }}\
"""
        # print(self.description)

    def _run(
        self,
        tool_input: str,
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> str:
        result = self.graphql_wrapper.run(tool_input)
        return json.dumps(result, indent=2)

    async def _arun(
        self,
        tool_input: str,
        run_manager: Optional[AsyncCallbackManagerForToolRun] = None,
    ) -> str:
        """Use the Graphql tool asynchronously."""
        raise NotImplementedError("GraphQLAPIWrapper does not support async")
    
def _get_graphql_tool(**kwargs: Any) -> BaseTool:
    graphql_endpoint = kwargs["graphql_endpoint"]
    wrapper = GraphQLAPIWrapper(graphql_endpoint=graphql_endpoint)
    return BaseGraphQLTool(graphql_wrapper=wrapper)

graphql_tool = _get_graphql_tool(graphql_endpoint="http://127.0.0.1:8000")